In [18]:
import numpy as np
import scipy as sp
import networkx as nx
import cupy as cp
import csv
from sklearn import preprocessing
from collections import OrderedDict
import time 

In [22]:
# Inputs to be given
dataset = 'BZR' # Choose the datasets among: 'PROTEINS', 'ENZYMES', 'BZR', 'COX2', 'DHFR', 'SYNTHETICnew'
iter_num = 3 # No: of WL iteration
ker = 1  # choose 0 for Gaussian kernel as base kernel and 1 for linear kernel
path_to_data_folder = ""

In [23]:
if dataset == 'PROTEINS':
    A = np.loadtxt(path_to_data_folder + "PROTEINS/PROTEINS_A.txt", dtype = int, delimiter=",", unpack=False)
    indicator = np.loadtxt(path_to_data_folder + "PROTEINS/PROTEINS_graph_indicator.txt", dtype = int, delimiter=",", unpack=False)
    NL = np.loadtxt(path_to_data_folder + "PROTEINS/PROTEINS_node_labels.txt", dtype = int, delimiter=",", unpack=False)
    NA = np.loadtxt(path_to_data_folder + "PROTEINS/PROTEINS_node_attributes.txt", dtype = float, delimiter=",", unpack=False)
    EL = None
    EA = None
    
if dataset == 'ENZYMES':
    A = np.loadtxt(path_to_data_folder + "ENZYMES/ENZYMES_A.txt", dtype = int, delimiter=",", unpack=False)
    indicator = np.loadtxt(path_to_data_folder + "ENZYMES/ENZYMES_graph_indicator.txt", dtype = int, delimiter=",", unpack=False)
    NL = np.loadtxt(path_to_data_folder + "ENZYMES/ENZYMES_node_labels.txt", dtype = int, delimiter=",", unpack=False)
    NA = np.loadtxt(path_to_data_folder + "ENZYMES/ENZYMES_node_attributes.txt", dtype = float, delimiter=",", unpack=False)
    EL = None
    EA = None
    
if dataset == 'BZR':
    A = np.loadtxt(path_to_data_folder + "BZR/BZR_A.txt", dtype = int, delimiter=",", unpack=False)
    indicator = np.loadtxt(path_to_data_folder + "BZR/BZR_graph_indicator.txt", dtype = int, delimiter=",", unpack=False)
    NL = np.loadtxt(path_to_data_folder + "BZR/BZR_node_labels.txt", dtype = int, delimiter=",", unpack=False)
    NA = np.loadtxt(path_to_data_folder + "BZR/BZR_node_attributes.txt", dtype = float, delimiter=",", unpack=False)
    EL = None
    EA = None
    
if dataset == 'COX2':
    A = np.loadtxt(path_to_data_folder + "COX2/COX2_A.txt", dtype = int, delimiter=",", unpack=False)
    indicator = np.loadtxt(path_to_data_folder + "COX2/COX2_graph_indicator.txt", dtype = int, delimiter=",", unpack=False)
    NL = np.loadtxt(path_to_data_folder + "COX2/COX2_node_labels.txt", dtype = int, delimiter=",", unpack=False)
    NA = np.loadtxt(path_to_data_folder + "COX2/COX2_node_attributes.txt", dtype = float, delimiter=",", unpack=False)
    EL = None
    EA = None
    
if dataset == 'DHFR':
    A = np.loadtxt(path_to_data_folder + "DHFR/DHFR_A.txt", dtype = int, delimiter=",", unpack=False)
    indicator = np.loadtxt(path_to_data_folder + "DHFR/DHFR_graph_indicator.txt", dtype = int, delimiter=",", unpack=False)
    NL = np.loadtxt(path_to_data_folder + "DHFR/DHFR_node_labels.txt", dtype = int, delimiter=",", unpack=False)
    NA = np.loadtxt(path_to_data_folder + "DHFR/DHFR_node_attributes.txt", dtype = float, delimiter=",", unpack=False)
    EL = None
    EA = None
    
if dataset == 'SYNTHETICnew':
    A = np.loadtxt(path_to_data_folder + "SYNTHETICnew/SYNTHETICnew_A.txt", dtype = int, delimiter=",", unpack=False)
    indicator = np.loadtxt(path_to_data_folder + "SYNTHETICnew/SYNTHETICnew_graph_indicator.txt", dtype = int, delimiter=",", unpack=False)
    NL = None
    NA = np.loadtxt(path_to_data_folder + "SYNTHETICnew/SYNTHETICnew_node_attributes.txt", dtype = float, delimiter=",", unpack=False)
    EL = None
    EA = None

In [25]:
if NL is not None:
    if len(NL.shape) == 1:
        NL = np.reshape(NL,(NL.shape[0],1))
if EL is not None:
    if len(EL.shape) == 1:
        EL = np.reshape(EL,(EL.shape[0],1))
if len(NA.shape) == 1:
    NA = np.reshape(NA,(NA.shape[0],1))

In [26]:
def create_data(A, indicator, NA, EA=None, NL=None, EL=None):
    U = np.unique(indicator)
    count=[]
    for i in range(U.shape[0]):
        tmp = indicator[indicator==U[i]]
        count.append(tmp.shape[0])
    adj=[]
    adj_list = []
    node_label = []
    node_attr = []
    edge_list = []
    edge_label = []
    edge_label_mat = []
    edge_attr= []
    penalty = 0
    p = 0
    for i in range(len(count)):  
        tmp = np.zeros((count[i], count[i]))
        tmp_E = np.zeros((count[i], count[i]))
        flag = 1
        breakpoint = np.sum(count[0:i+1]) 
        tmp_EA = OrderedDict()
        while flag==1:
            if A[p,0]<=breakpoint and A[p,1]<=breakpoint:
                A[p,0] = A[p,0] - 1
                A[p,1] = A[p,1] - 1
                tmp[A[p,0]-penalty,A[p,1]-penalty] = 1
                
                if EL is not None: 
                    tmp_E[A[p,0]-penalty,A[p,1]-penalty] = int(EL[p])
                else:
                    tmp_E[A[p,0]-penalty,A[p,1]-penalty] = int(1)
                    
                if EA is not None:  
                    tmp_EA[str(A[p,0]-penalty)+str(A[p,1]-penalty)] = EA[p,:]
                p=p+1 
                if p == A.shape[0]:
                    adj_list_tmp = []
                    for j in range(tmp.shape[0]):
                        adj_list_tmp.append(np.nonzero(tmp[j,:]))
                        
                    indx = np.nonzero(np.triu(tmp))   
                    tmp1 = np.array((indx[0],indx[1]))
                    edge_list.append(tmp1.T)
                    
                    if NL is not None:
                        node_label.append(NL[penalty:penalty+breakpoint,0])
                    else:
                        node_label.append(np.ones(([breakpoint,1])))
                    node_attr.append(NA[penalty:penalty+breakpoint,:])
                    adj_list.append(adj_list_tmp)
                    adj.append(sp.sparse.csr_matrix(tmp)) 
                    edge_label.append(tmp_E[indx[0],indx[1]])
                    edge_label_mat.append(tmp_E)
                    if EA is not None:
                        edge_attr.append(tmp_EA) 
                    del tmp, tmp_E
                    flag=0
                    
            else:
                
                adj_list_tmp = []
                for j in range(tmp.shape[0]):
                    adj_list_tmp.append(np.nonzero(tmp[j,:]))
                    
                
                indx = np.nonzero(np.triu(tmp)) 
                tmp1 = np.array((indx[0],indx[1]))
                edge_list.append(tmp1.T)
                
                if NL is not None:
                    node_label.append(NL[penalty:penalty+breakpoint,0])
                else:
                    
                    node_label.append(np.ones(([breakpoint,1])))
                node_attr.append(NA[penalty:penalty+breakpoint,:])
                adj_list.append(adj_list_tmp)
                adj.append(sp.sparse.csr_matrix(tmp)) 
                edge_label.append(tmp_E[indx[0],indx[1]])
                edge_label_mat.append(tmp_E)
                if EA is not None:
                        edge_attr.append(tmp_EA) 
                del tmp, tmp_E
                flag = 0 
        flag = 1
        penalty = breakpoint  
    return adj, adj_list, node_label, node_attr, edge_list, edge_label,edge_label_mat, edge_attr

In [27]:
adj,adj_list, NL_list, NA_list, E_list, EL_list,ELM_list, EA_list = create_data(A, indicator, NA,EA, NL, EL)

In [28]:
def WL_refinement(adj_list, NL_list):
    WL_list=[]
    od = OrderedDict()
    p = 0
    for i in range(0, len(adj_list)):
        WL_tmp=[]
        for j in range(0,len(adj_list[i])):
            tmp = np.sort(NL_list[i][adj_list[i][j]])
            tmp = ''.join(str(x) for x in tmp) 
            label = str(NL_list[i][j])+tmp
            if label in od:
                WL_tmp.append(od[label])
            else:
                od[label] = p
                WL_tmp.append(p)
                p+=1
        WL_list.append(np.asarray(WL_tmp))
    return WL_list,p

In [29]:
def WL_labeling_SP(SP, ELM_list, WL_list):
    WL_sp_add = []
    for i in range(len(SP)):
        ELM = ELM_list[i]
        od = OrderedDict()
        for j in range(len(SP[i])):
            path = SP[i][j]  
            pathl = WL_list[i][path] 
            pathl = pathl.tolist()
            epath=[]
                        
            for n in range(len(pathl)-1):
                epath.append(int(ELM[path[n],path[n+1]])) 
                
            label1 = pathl + epath
            pathl.reverse()
            epath.reverse() 
            label2 = pathl + epath
            
            label1, label2 = ' '.join(str(x) for x in label1) , ' '.join(str(x) for x in label2) 
            if label1 not in od and label2 not in od:     
                od[label1] = []    
            if label1 in od:  
                od[label1].append([path[0], path[-1]]) 
            if label2 in od and label2!=label1:
                path.reverse()
                od[label2].append([path[0], path[-1]])           
 
        WL_sp_add.append(od)
    return WL_sp_add

In [30]:
def pairwise_computation(common_keys,od1,od2, NA1, NA2, odEA1, odEA2, ker, beta1, beta2): 
                tmp11, tmp12, tmp21, tmp22 = np.zeros((0)), np.zeros((0)), np.zeros((0)), np.zeros((0))
                tmpE1, tmpE2 = np.zeros((0,beta2)), np.zeros((0,beta2)) 
                for k in common_keys:
                    lst1,lst2 = od1[k], od2[k]  
                     
                    if odEA1 != None:
                        EAlst1, EAlst2= [],[] 
                        for m in range(len(lst1)):
                            c = str(lst1[m][0])+str(lst1[m][1])
                            EAlst1.append(odEA1[c])
                        for m in range(len(lst2)):
                            c = str(lst2[m][0])+str(lst2[m][1])
                            EAlst2.append(odEA2[c])
                        EAlst1=np.asarray(EAlst1)
                        EAlst2=np.asarray(EAlst2) 
                        EAlst1, EAlst2 = np.repeat(EAlst1, len(EAlst2), axis=0), np.tile(EAlst2,(len(EAlst1),1))   
                        tmpE1 = np.concatenate((tmpE1,EAlst1), axis=0)
                        tmpE2 = np.concatenate((tmpE2,EAlst2), axis=0)
                        
                    lst1,lst2 = np.asarray(lst1, dtype = np.int32), np.asarray(lst2, dtype = np.int32)  
                    lst1, lst2 = np.repeat(lst1, len(lst2), axis=0), np.tile(lst2,(len(lst1),1))   
                    tmp11 = np.concatenate((tmp11,lst1[:,0]), axis=0)
                    tmp12 = np.concatenate((tmp12,lst2[:,0]), axis=0)              
                    tmp21 = np.concatenate((tmp21,lst1[:,1]), axis=0)
                    tmp22 = np.concatenate((tmp22,lst2[:,1]), axis=0) 
                
                if len(common_keys)!=0: 
                    N11, N12 = NA1[tmp11.astype(int)], NA2[tmp12.astype(int)]   
                    N21, N22 = NA1[tmp21.astype(int)], NA2[tmp22.astype(int)]  
                    if ker==0:
                        t1 =  np.exp(-1/beta1*(np.linalg.norm((N11-N12),axis=1)**2)) 
                        t2 =  np.exp(-1/beta1*(np.linalg.norm((N21-N22),axis=1)**2))  
                        t = np.multiply(t1, t2)   
                        if len(EA_list)!=0:
                            t = np.multiply(t, np.exp(-1/beta2*(np.linalg.norm((tmpE1-tmpE2),axis=1)**2)))
                        t = np.sum(t) 
                         
                    else:
                        t1 =  np.multiply(N11,N12) 
                        t1 = np.sum(t1,axis = 1)
                        t2 =  np.multiply(N21,N22) 
                        t2 = np.sum(t2,axis = 1)  
                        t = np.multiply(t1, t2)
                        if odEA1 != None:
                            t1 = np.multiply(tmpE1, tmpE2)
                            t1= np.sum(t1, axis = 1)
                            t = np.multiply(t, t1)
                        t = np.sum(t)  
                else:
                    t=0 
                return t

In [32]:
def NP_kernel_imp2(adj, adj_list, NL_list, NA_list, E_list, EL_list,ELM_list, EA_list, h, ker): 
     
    beta1 = NA_list[0].shape[1]
    beta2= 1
    if len(EA_list)!=0:
        beta2 = EA.shape[1]
    
    
#     # Finding WL edge addresses
    WL_list, label_count = WL_refinement(adj_list, NL_list)

    
    # Finding shortest paths
    SP = [] 
    for j in range(len(adj_list)): 
        sp = []
        G = nx.from_scipy_sparse_matrix(adj[j]) 
        for k in range(len(adj_list[j])):
            for m in range(len(adj_list[j])):
                if m>k: 
                    if nx.has_path(G,k,m):
                        path = nx.shortest_path(G,k,m) 
                        sp.append(path)
        SP.append(sp) 
        
        
    # Finding out the WL labels and WL labels for shortest paths for the whole iteration and the feature dictionary
    WL_label_list=[WL_list]
    label_count_list = [label_count]
    curr_list = WL_list
    SP_WL_label_list = []
    SP_WL_label_list.append(WL_labeling_SP(SP, ELM_list, WL_list))
    for i in range(1,h):
        WL_list_new, label_count_new = WL_refinement(adj_list, WL_label_list[-1])
        WL_label_list.append(WL_list_new)
        label_count_list.append(label_count_new)
        SP_WL_label_list.append(WL_labeling_SP(SP, ELM_list, WL_label_list[-1]))                
    K = np.zeros((len(adj_list), len(adj_list))) 
    odEA1 , odEA2 = None, None
    for i in range(len(adj_list)):  
        NA1 = NA_list[i]
        if len(EA_list)!=0:
                odEA1 = EA_list[i] 
        for j in range(len(adj_list)):
            
            if i<=j:
                if len(EA_list)!=0:
                    odEA2 = EA_list[j]  
                kv1=0
                NA2 = NA_list[j]
                for k in range(h): 
                    kv = 0
                    sp_od1, sp_od2 = SP_WL_label_list[k][i], SP_WL_label_list[k][j]
                    st1, st2 = set(sp_od1), set(sp_od2) 
                    common_keys = st1.intersection(st2)
                    if len(common_keys)!=0: 
                        kv = pairwise_computation(common_keys, sp_od1,sp_od2, NA1, NA2, odEA1, odEA2, ker, beta1, beta2)  
                    kv1+=kv 
                K[i,j], K[j,i] = kv1,kv1   
    return K

In [33]:
start = time.time()

K_NPS = NP_kernel_imp2(adj, adj_list, NL_list, NA_list, E_list, EL_list,ELM_list, EA_list, iter_num, ker) 
end = time.time()

In [14]:
print("Time taken (in seconds): {}".format(end - start))

246.3698341846466